In [1]:
data_var = '2024-02-05'

In [2]:
import pandas as pd; pd.set_option('display.max_columns', None)
import os
from pycaret.classification import *
from scripts.PreparaDataset import *
from scripts.TeoriaRetornos import *

### Funções

In [3]:
def atualiza_dia(_jogos):
    _jogos = prepara_df(_jogos)

    _jogos = _jogos.reset_index(drop=True)

    return _jogos

### Pegando os jogos do dia

In [4]:
df_today = pd.read_csv(f'jogos_do_dia/{data_var}.csv')

### Atualizando os datasets que tem jogo hoje

In [5]:
temporada = pd.read_csv('data/Entire_Season.csv')
temporada = temporada[temporada['Date'] < data_var]
temporada = pd.concat([temporada, df_today])
ligas = df_today['League'].unique().tolist()
temporada = temporada[temporada['League'].isin(ligas)]
temporada.reset_index(drop=True, inplace=True)

### Cria variáveis na temporada e nos jogos do dia

In [6]:
games = atualiza_dia(temporada)
temporada = games[games['Date'] < data_var]
os.makedirs(f'apostas_do_dia/{data_var}', exist_ok=True)

In [7]:
jogos = games[games['Date'] == data_var]

jogos = jogos.drop(columns=['Home_Pts', 'Away_Pts',
       'Back_Home', 'PL_Home', 'Back_Away',
       'PL_Away', 'Back_Over', 'PL_Over', 'PL_Under', 'Back_HA_H', 'PL_HA_H',
       'PL_HA_A', ])

jogos

,Date,League,Time,Home,Away,Odds_H,Odds_A,Over_Line,Odds_Over,Odds_Under,HA_Line,HA_Odds_H,HA_Odds_A,Fixture ID,P(H),P(A),P(O),P(U),P_Diff,Porc_Over_Home,Porc_Over_Away,CustoGolHome,CustoGolAway,Avg_CG_H,DP_CG_H,CV_CG_H,Avg_CG_A,DP_CG_A,CV_CG_A,MediaCustoGolHome,MediaCustoGolAway,Last_CG_H,Last_CG_A,Acima_Last_CG_H,Acima_Last_CG_A,Abaixo_Last_CG_H,Abaixo_Last_CG_A,CV_ML,CV_Over,CV_HA,Retornos_BH_Acu,Avg_Retornos_BH,Custo_Retorno_BH,Avg_Porc_BH_Bookie,Avg_Porc_BH_Real,Dist_Porc_BH,Retornos_BA_Acu,Avg_Retornos_BA,Custo_Retorno_BA,Avg_Porc_BA_Bookie,Avg_Porc_BA_Real,Dist_Porc_BA
11780,2024-02-05,Brasil Nbb,19:30,Pato,São José,1.57,2.31,154.5,1.85,1.85,-4.5,2.01,1.72,bXXc8UxI,0.636943,0.432900,0.540541,0.540541,0.069843,0.6,0.6,NaN,NaN,222.882,184.206669,0.826476,147.918,48.103160,0.325202,209.892,180.520,131.92,187.68,0.0,0.0,0.0,0.0,0.269721,0.000000,0.109952,4.92,0.984,0.579268,0.477690,0.6,0.122310,-2.85,-0.570,-2.298246,0.550342,0.5,-0.050342
11781,2024-02-05,Brasil Nbb,19:30,União Corinthians,Cerrado,1.48,2.53,150.5,1.85,1.85,-5.5,2.01,1.72,jTeRj27p,0.675676,0.395257,0.540541,0.540541,0.070933,0.4,0.6,NaN,NaN,346.894,214.276159,0.617699,204.298,51.488685,0.252027,229.984,238.376,128.77,222.88,0.0,0.0,0.0,0.0,0.370305,0.000000,0.109952,-3.37,-0.674,-0.712166,0.492638,0.3,-0.192638,-5.00,-1.000,-1.530000,0.251118,0.1,-0.151118
11782,2024-02-05,Eua Nba,21:00,Charlotte Hornets,Los Angeles Lakers,5.07,1.19,230.5,1.83,1.91,9.5,2.01,1.76,p4twIhGI,0.197239,0.840336,0.546448,0.523560,0.037575,0.6,0.8,NaN,NaN,410.768,197.514935,0.480843,329.492,105.230642,0.319372,343.648,341.846,296.24,294.93,0.0,0.0,0.0,0.0,0.876541,0.030251,0.093781,-3.44,-0.688,-5.915698,0.327973,0.1,-0.227973,-0.21,-0.042,-4.523810,0.406747,0.3,-0.106747
11783,2024-02-05,Eua Nba,21:00,Cleveland Cavaliers,Sacramento Kings,1.44,2.94,232.5,1.85,1.93,-6.5,2.02,1.77,dt960Qpo,0.694444,0.340136,0.540541,0.518135,0.034580,0.4,0.4,NaN,NaN,283.998,105.797636,0.372530,199.758,52.062012,0.260625,219.104,229.316,248.98,259.53,0.0,0.0,0.0,0.0,0.484320,0.029930,0.093286,4.15,0.830,0.530120,0.608101,0.8,0.191899,0.19,0.038,51.052632,0.553981,0.6,0.046019
11784,2024-02-05,Eua Nba,21:30,Atlanta Hawks,Los Angeles Clippers,2.30,1.66,244.5,1.83,1.93,1.5,2.08,1.68,lzdhOTxh,0.434783,0.602410,0.546448,0.518135,0.037192,0.8,0.6,NaN,NaN,270.326,48.320599,0.178749,209.626,99.867990,0.476410,238.900,209.626,242.52,160.48,0.0,0.0,0.0,0.0,0.228560,0.037612,0.150448,0.41,0.082,15.853659,0.610584,0.6,-0.010584,1.99,0.398,1.658291,0.605628,0.6,-0.005628
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11843,2024-02-05,Uruguai Liga Uruguaia,21:00,Goes,Urupan,1.63,2.19,160.5,1.85,1.85,-3.5,1.97,1.74,GYaXLu00,0.613497,0.456621,0.540541,0.540541,0.070118,0.4,0.2,NaN,NaN,258.358,62.200263,0.240752,254.370,173.332806,0.681420,154.260,268.078,249.38,132.88,0.0,0.0,0.0,0.0,0.207319,0.000000,0.087674,-0.20,-0.040,-15.750000,0.000000,0.0,0.000000,-1.88,-0.376,-3.164894,0.000000,0.0,0.000000
11844,2024-02-05,Irã Superliga,09:30,Limondis,Mes Rafsanjan,1.70,2.05,162.5,1.83,1.87,-6.5,2.00,1.69,MFW7V0Hn,0.588235,0.487805,0.546448,0.534759,0.076040,0.0,0.0,NaN,NaN,277.626,80.977629,0.291679,228.420,121.115439,0.530231,0.000,0.000,0.00,102.60,0.0,0.0,0.0,0.0,0.131993,0.015289,0.118809,0.00,0.000,inf,0.000000,0.0,0.000000,0.00,0.000,inf,0.000000,0.0,0.000000
11845,2024-02-05,Israel Liga Leumit,14:30,Ironi Nahariya,Maccabi Rehovot,1.65,2.15,166.5,1.80,1.88,-2.5,1.85,1.85,fLjM0LuQ,0.606061,0.465116,0.555556,0.531915,0.071177,0.0,0.0,NaN,NaN,156.936,43.518779,0.277303,135.850,32.919264,0.242321,0.000,0.000,208.55,131.12,0.0,0.0,0.0,0.0,0.186081,0.030744,0.000000,0.00,0.000,inf,0.000000,0.0,0.000000,0.00,0.000,inf,0.000000,0.0,0.000000
11846,2024-02-05,Rússia Superliga,10:00,Temp Sumz Revda,Bars Rostov,1.15,5.03,149.5,1.87,1.83,-13.5,2.09,1.63,dE8W19wN,0.86

## Apostas do dia

### Over v0 (LDA)

In [8]:
# Duplicando o dataset
df_over_v0 = jogos.copy()

df_over_v0.reset_index(drop=True, inplace=True)
df_over_v0.replace([np.inf, -np.inf], 0, inplace=True)

if not df_over_v0.empty:
    mdl_over_v0 = load_model('ML/over/v1/v1_over', verbose=False)
    prev = predict_model(mdl_over_v0, df_over_v0)
    prev = prev[prev['prediction_label'] ==  1]
    prev.reset_index(drop=True, inplace=True)
    
    # Filtros pós modelo

    if not prev.empty:
        prev.to_csv(f'apostas_do_dia/{data_var}/over_v0_{data_var}.csv', index=False)

        print('------------ MÉTODO OVER V0 (LDA) ------------')
        print()

        prev['Bet'] = 'Over'
        ligs = prev['League'].unique().tolist()

        display(prev[['Time', 'League', 'Home', 'Away', 'Over_Line', 'Odds_Over', 'prediction_score', 'Bet']])
        print()
    else: print('Sem jogos')
else: print('Sem jogos')

------------ MÉTODO OVER V0 (LDA) ------------



,Time,League,Home,Away,Over_Line,Odds_Over,prediction_score,Bet
0,21:30,Eua Nba,Brooklyn Nets,Golden State Warriors,234.5,1.82,1.0,Over
